In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits.basemap import Basemap
from math import hypot

# функція яка потрібна для малювання кола (використовується потім в циклі)
def circle(alpha, l0, b0):
    alpha = np.radians(alpha)
    l0 = np.radians(l0)
    b0 = np.radians(-(90 - b0))
    t = np.linspace(0, 2 * np.pi, 400)
    x = np.sin(alpha) * np.cos(b0) * np.cos(l0) * np.cos(t) + np.sin(alpha) * np.sin(l0) * np.sin(t) - np.cos(alpha) * np.sin(b0) * np.cos(l0)
    y = -np.sin(alpha) * np.cos(b0) * np.sin(l0) * np.cos(t) + np.sin(alpha) * np.cos(l0) * np.sin(t) + np.cos(alpha) * np.sin(b0) * np.sin(l0)
    z = np.sin(alpha) * np.sin(b0) * np.cos(t) + np.cos(alpha) * np.cos(b0)
    l = -np.arctan2(y, x)
    b = np.arcsin(z)
    return [l * 180 / np.pi, b * 180 / np.pi]



# для будь якого типу подій які наносяться на карту, створюється по два відповідних нампай массива (довгота широта)
init_evs_l = np.array([])
init_evs_b = np.array([])

circle_l = np.array([30])
circle_b = np.array([0])
circle_r = np.array([10]) # радіус відповідного кола



# створює карту неба, проекція 'moll'
m = Basemap(projection='moll', lon_0=0, celestial=True, resolution='c')
# малює на ній паралелі і меридіани
m.drawparallels(np.arange(-90., 90., 15.), labels=[1, 0, 0, 0], color='k')
m.drawmeridians(np.arange(0., 360., 30.))
meridian = np.arange(0., 360., 30.)
for i in np.arange(len(meridian)):
    plt.annotate(np.str(meridian[i]) + '$^\\circ$', xy=m(meridian[i], 0), xycoords='data')


    
# наносить на небо положення обєктів з координатами довгота = init_evs_l, широта = init_evs_b  
l0, b0 = m(init_evs_l, init_evs_b)
m.scatter(l0, b0, s=10, c='k', marker='o', label='CRs E > 1e20 eV')

# наносить положення інших обєктів і так далі, для кожного типу обєктів що б нанести на небо треба такі два рядки
#l2, b2 = m(init_evs_l, init_evs_b)
#m.scatter(l0, b0, s=10, c='k', marker='o', label='CRs E > 1e20 eV')



# алгоритм який будує кола, з центром в (circle_l[i], circle_b[i])
for i in range(len(circle_l)):
    r = circle_r[i] # радіус кола
    res = circle(r, circle_l[i], circle_b[i]) # зсув 90 градусів вже враховано всередині функціїї
                                                                       # circ(). підставляємо просто b а не 90-b
    # Розбиваємо кожне коло на дві частини: 0 < l < 180 і l > 180
    # для того, що не малювалися лінії, які з'єднують точки
    # через всю карту
    res_left = [[], []]
    res_right = [[], []]
    for i in range(len(res[0])):
        if 0 < res[0][i] < 180:
            res_left[0].append(res[0][i])
            res_left[1].append(res[1][i])
        else:
            res_right[0].append(res[0][i])
            res_right[1].append(res[1][i])
    # Кожну з двох частин кола будемо малювати відрізками між двома сусідніми точками,
    # за умови, що відстань між ними менше певної величини. Ця величина залежить від загальної кількості
    # точок кола, яка задається всередині функції circle(): t = np.linspace(0, 2 * np.pi, 700) (коло має 700 точок)
    for i in range(len(res_left[0]) - 1):
        if hypot(res_left[0][i + 1] - res_left[0][i], res_left[1][i + 1] - res_left[1][i]) < 10:
            x_l, y_l = m(res_left[0][i], res_left[1][i])
            x_l2, y_l2 = m(res_left[0][i + 1], res_left[1][i + 1])
            m.plot([x_l, x_l2], [y_l, y_l2], color='b', ls='-', lw=1, label='z=6 + extragal')
    for i in range(len(res_right[0]) - 1):
        if hypot(res_right[0][i + 1] - res_right[0][i], res_right[1][i + 1] - res_right[1][i]) < 10:
            x_r, y_r = m(res_right[0][i], res_right[1][i])
            x_r2, y_r2 = m(res_right[0][i + 1], res_right[1][i + 1])
            m.plot([x_r, x_r2], [y_r, y_r2], color='b', ls='-', lw=1)

            
# _________Додавання легенди до малюнка (з видалленям дублікатів однакових ліній)_________
handles, labels = plt.gca().get_legend_handles_labels()
newLabels, newHandles = [], []
for handle, label in zip(handles, labels):
    if label not in newLabels:
        newLabels.append(label)
        newHandles.append(handle)
plt.legend(newHandles, newLabels, loc='upper right', prop={'size': 12})


plt.tight_layout()

plt.show()

